# interpolar sexo

In [22]:
import pandas as pd
from scipy.interpolate import interp1d
import numpy as np

# cargar archivos
file_path = "data/eph/Sexo_concatenado.csv"
sexo = pd.read_csv(file_path)
sexo["Año"] = pd.to_datetime(sexo["Año"], format="%Y")

varon = sexo[sexo["Sexo"] == "Varón"]
mujer = sexo[sexo["Sexo"] == "Mujer"]
varon_2015 = pd.DataFrame({"Año": [pd.to_datetime('2015')], "Sexo": ["Varón"]})
mujer_2015 = pd.DataFrame({"Año": [pd.to_datetime('2015')], "Sexo": ["Mujer"]})

varon = pd.concat([varon, varon_2015])
mujer = pd.concat([mujer, mujer_2015])

varon = varon.sort_values(by="Año").reset_index(drop=True)
mujer = mujer.sort_values(by="Año").reset_index(drop=True)

columnas = sexo.columns[2:]

for columna in columnas:
    valid_varon = varon.dropna(subset=[columna])
    valid_mujer = mujer.dropna(subset=[columna])
    
    if len(valid_varon) > 1 and len(valid_mujer) > 1:
        varon_interp = interp1d(valid_varon["Año"].astype(np.int64), valid_varon[columna], kind='linear', fill_value="extrapolate")
        mujer_interp = interp1d(valid_mujer["Año"].astype(np.int64), valid_mujer[columna], kind='linear', fill_value="extrapolate")

        varon[columna] = varon_interp(varon["Año"].astype(np.int64))
        mujer[columna] = mujer_interp(mujer["Año"].astype(np.int64))
    
sexo = pd.concat([varon, mujer]).sort_values(by=["Sexo","Año"]).reset_index(drop=True)

sexo.to_csv("data/eph/Sexo_concatenado_interp.csv", index=False)

# Interpolar región

In [31]:
import pandas as pd
from scipy.interpolate import interp1d
import numpy as np

# cargar archivos
file_path = "data/eph/Region_concatenado.csv"
reg = pd.read_csv(file_path)

DFs = {}
regiones = reg["REGION"].unique()
for region in regiones:
    region_df = reg[reg["REGION"] == region].copy()
    region_df["Año"] = pd.to_datetime(region_df["Año"], format="%Y")
    region_2015 = pd.DataFrame({"Año": [pd.to_datetime('2015', format="%Y")], "REGION": [region]})
    region_df = pd.concat([region_df, region_2015])
    region_df = region_df.sort_values(by="Año").reset_index(drop=True)
    DFs[region] = region_df
    
columnas = reg.columns[2:]

for columna in columnas:
    for region in regiones:
        valid = DFs[region].dropna(subset=[columna])
        if len(valid) > 1:
            interp = interp1d(valid["Año"].astype(np.int64), valid[columna], kind='linear', fill_value="extrapolate")
            DFs[region][columna] = interp(DFs[region]["Año"].astype(np.int64))
            
reg_interp = pd.concat(DFs.values()).sort_values(by=["REGION","Año"]).reset_index(drop=True)
reg_interp.to_csv("data/eph/Region_concatenado_interp.csv", index=False)